In [1]:
import pandas as pd
import re
import xml.etree.ElementTree as ET

# converts string to a standardised array [firstname, lastname]
def standardise(name):
    name = name.replace("Dr.","").replace("Prof.","").replace(" von", "").replace(u'\xa0'+"von","").replace("von ", "").replace("von"+u'\xa0',"").replace(u'\xa0',' ').lower().replace('ä','ae').replace('ö','oe').replace('ü','ue').replace('ß','ss')

    firstname = ""
    lastname = ""
    if ',' in name:
        name = name.split(',')
        firstname = name[-1]
        for string in firstname.split(' '):
            if len(string) > 0:
                firstname = string
                break
        lastname = name[0]
    else:
        name = name.split(' ')
        for string in name:
            if len(string) > 0:
                if len(firstname) == 0:
                    firstname = string
                else:
                    lastname = string
        if len(lastname) == 0:
            lastname = firstname
            firstname = ""
    if len(firstname) == 0:
        firstname = "?"
    return [firstname, lastname]

# tries to find deputies in a string, returns a list with the indizes of deputies found
def find_deputies(string, df):
    string = string.lower().replace('ä','ae').replace('ö','oe').replace('ü','ue').replace('ß','ss')
    df = df.apply(lambda row: row['firstname'] in string and row['lastname'] in string, axis = 1)
    return df[df].index.tolist()

# tries to find parties in a string, returns a list with the indizes of parties found
def find_parties(string):
    #remove parties in [...], these just describe a deputy:
    string = re.sub(r'\[[^\]]+\]', '', string)
    possible_parties = [['CDU','CSU'], ['SPD'], ['GRÜNE'], ['FDP'], ['AFD'], ['LINKE']]
    parties = []
    #Search parties:
    for i in range(len(possible_parties)):
        for party_name in possible_parties[i]:
            if party_name in string.upper():
                parties.append(i)
                break
    return parties

# tries to interpret a comment in plenar protocol, returns the the event(s) and links to deputies and parties
def interpret_comment(comment, df_deputies, comment_id, speech_id):
    # Arrays to save Series wich can be added to the data_frame
    events = []
    deputy_links = []
    party_links = []

    possible_types = ['Beifall', ': ', 'Lachen', 'Zuruf', 'Widerspruch', 'Heiterkeit']

    comment = comment.replace(u'\xa0',' ').replace('–','-')

    # Multiple events are seperated by ' - ':
    c = 0
    for single_comment in comment.split(' - '):
        event_id = str(comment_id)+"_"+str(c)

        #Search parties:
        parties = find_parties(single_comment)

        #Search deputies:
        deputies = find_deputies(single_comment, df_deputies)

        # search event types:
        event_type = 0
        for possible_type in possible_types:
            if possible_type in single_comment:
                break;
            event_type += 1

        event = pd.Series([event_type, speech_id], name = event_id, index = ['event_type', 'speech_id'])
        events.append(event)

        for party in parties:
            party_link = pd.Series([party,event_id], index = ['party_id', 'event_id'])
            party_links.append(party_link)

        for deputy in deputies:
            deputy_link = pd.Series([deputy, event_id], index = ['deputy_id','event_id'])
            deputy_links.append(deputy_link)

        c += 1
    return [events, deputy_links, party_links]


# Loads a file, e.g. MDB_STAMMDATEN.XML. Returns a df of the deputies of Bundestag and a second df with their periods
def load_deputies_from_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()
    person_row_list = []

    link_list = []

    counter = 0
    for child in root.findall('MDB'):
        item = child.find('ID')
        person_id = "" if item is None else int(item.text)
        item = child.find('./NAMEN/NAME/VORNAME')
        firstname = "" if item is None else item.text
        item = child.find('./NAMEN/NAME/NACHNAME')
        lastname = "" if item is None else item.text
        item = child.find('./NAMEN/NAME/AKAD_TITEL')
        titel = "" if item is None else item.text
        item = child.find('./BIOGRAFISCHE_ANGABEN/GEBURTSORT')
        geburtsort = "" if item is None else item.text
        item = child.find('./BIOGRAFISCHE_ANGABEN/GEBURTSDATUM')
        geburtsdatum = "" if item is None else item.text
        item = child.find('./BIOGRAFISCHE_ANGABEN/STERBEDATUM')
        sterbedatum = "" if item is None else item.text
        item = child.find('./BIOGRAFISCHE_ANGABEN/BERUF')
        berufe = "" if item is None or item.text is None else item.text.replace(' /',',').replace('/',',').replace(' und ',',').replace(', ',',').split(',')
        item = child.find('./BIOGRAFISCHE_ANGABEN/PARTEI_KURZ')
        print(item.text)
        fraktion = "" if item is None or item.text is None or  len(find_parties(item.text)) < 1 else find_parties(item.text)[0]

        for wahlperiode in child.findall('./WAHLPERIODEN/WAHLPERIODE'):
            link_list.append(pd.Series([int(wahlperiode.find('WP').text),person_id], index = ['election_period', 'deputy_id']))

        [firstname,lastname] = standardise(lastname+','+firstname)
        person_row_list.append(pd.Series([firstname, lastname,titel,geburtsort,geburtsdatum,sterbedatum,fraktion,berufe], name = person_id, index = ['firstname','lastname', 'title', 'birthplace', 'birthdate', 'deathday', 'fraktion', 'jobs']))
        counter += 1
        if counter % 500 == 0:
            print(f'Fetched {counter} deputies...')

    df_deputies = pd.DataFrame(person_row_list)
    dl2 = pd.DataFrame(link_list)

    return [df_deputies, dl2]

# loads a file, e.g. plenarprotokoll_xxxx.xml. Returns [session_row, dl_session_members, df_events], where all elements needs to be append
# to the general dataframe
def load_session_from_xml(path, df_deputies, dl_deputies_periods):
    # Read Session meta-data and create a pd.Series containing that information:
    session_id = int(path.split("_")[-1].split('.')[0])
    tree = ET.parse(path)
    root = tree.getroot()
    session_number = int(root.find('./vorspann/kopfdaten/plenarprotokoll-nummer/sitzungsnr').text)
    election_period = int(root.find('./vorspann/kopfdaten/plenarprotokoll-nummer/wahlperiode').text)
    date = root.find('./vorspann/kopfdaten/veranstaltungsdaten/datum').attrib['date']
    session_row = pd.Series([election_period,session_number,date], name=session_id, index = ['election_period','session_number','date'])

    # Create a list of participants
    # All deputies of this period are there, only the 'entschuldigte Abgeordnete' are missing
    df_participants = df_deputies.loc[dl_deputies_periods[dl_deputies_periods.election_period == election_period]['deputy_id']] # All deputies of this election period
    entschuldigte_anlage = root.find('./anlagen/anlage/anlagen-text[@anlagen-typ="Entschuldigte Abgeordnete"]')
    missing_deputies = []
    if entschuldigte_anlage is not None:
        for tr in entschuldigte_anlage.findall('./table/tbody/tr'):
            name = tr.find('td').text
            missing_deputies += find_deputies(name,df_participants)
    df_participants = df_participants.drop(missing_deputies) # drop deputies mentioned in "Entschuldigte Abgeordnete"
    dl_session_members = pd.DataFrame(list(zip([session_id]*len(df_participants), df_participants.index.tolist()))) # Create data link
    dl_session_members.columns = ['session_id', 'deputy_id']
    # TODO: former deputies of this election period will still appear, maybe filter them by bundestag membership period?

    # Create list of all events in this session
    events = [] # Data Series of events
    deputy_event_links = [] # Data Series of deputy_event - Links
    party_event_links = [] # Data Series of party event links
    speeches = [] # Data Series containing information about speeches

    comment_id = 0 # counts comments in this session
    for tagesordnungspunkt in root.findall('./sitzungsverlauf/tagesordnungspunkt'):
        for kommentar in tagesordnungspunkt.findall('kommentar'):
            [comment_events, comment_deputy_links, comment_party_links] = interpret_comment(kommentar.text, df_participants, str(session_id) + "_" + str(comment_id), 0) # use df_participants to reduce apearance of name duplications
            events += comment_events
            deputy_event_links += comment_deputy_links
            party_event_links += comment_party_links
            comment_id += 1
        speech_list = tagesordnungspunkt.findall('rede')
        for i in range(len(speech_list)):
            # Speech id of actual speech:
            speech_id = speech_list[i].attrib['id']
            # Speaker in this speech:
            speaker_id = speech_list[i].find('p[@klasse="redner"]/redner').attrib['id']
            speeches.append(pd.Series([speaker_id, session_id], name = speech_id, index = ['speaker_id','session_id']))
            # Speech_id of next speech (if available):
            next_speech_id = speech_list[i+1].attrib['id'] if i+1 < len(speech_list) else speech_id
            comment_list = speech_list[i].findall('kommentar')
            for j in range(len(comment_list)):
                if j < len(comment_list) - 1:
                    [comment_events, comment_deputy_links, comment_party_links] = interpret_comment(comment_list[j].text, df_participants, str(session_id) + "_" + str(comment_id), speech_id) # use df_participants to reduce apearance of name duplications
                    events += comment_events
                    deputy_event_links += comment_deputy_links
                    party_event_links += comment_party_links
                else: # Last comment in a speech is usually the applause for the next speeker
                    [comment_events, comment_deputy_links, comment_party_links] = interpret_comment(comment_list[j].text, df_participants, str(session_id) + "_" + str(comment_id), next_speech_id) # use df_participants to reduce apearance of name duplications
                    events += comment_events
                    deputy_event_links += comment_deputy_links
                    party_event_links += comment_party_links
                comment_id += 1



    return[session_row, dl_session_members, events, deputy_event_links, party_event_links, speeches]

#loads all databases by fetching them from original files
def fetch_all(mdb_path, plenar_paths):

    df_deputies = None # DF with deputies: ['firstname','lastname', 'title', 'birthplace', 'birthdate', 'deathday', 'fraktion', 'jobs'], index = deputy_id
    dl_deputy_periods = None # DL for deputies and election_period: ['election_period', 'deputy_id']
    df_sessions = pd.DataFrame() # DF with parliament session information: ['election_period','session_number','date'], index = session_id
    dl_session_deputy = pd.DataFrame() # DL for linking parliament sessions to deputies: ['session_id', 'deputy_id']
    df_events = pd.DataFrame() # DF with events during sessions: ['event_type', 'speech_id'], index = ignore
    df_speeches = pd.DataFrame() # DF with speeches: ['speaker_id', 'session_id']
    dl_deputy_event = pd.DataFrame() # DL for linking deputies to events they participated in: ['deputy_id','event_id'], index = ignore
    dl_party_event = pd.DataFrame() # DL for linking parties to events they participate in: ['party_id', 'event_id'], index = ignore

    print(f'Fetching {mdb_path} ...')
    [df_deputies, dl_deputy_period] = load_deputies_from_xml(mdb_path)

    counter = 1

    for path in plenar_paths:
        print(f'Fetching no {counter}: {path} ...')
        counter += 1
        [session_row, dl_session_members, events, deputy_event_links, party_event_links, speeches] = load_session_from_xml(path, df_deputies, dl_deputy_period)
        df_sessions = pd.concat([df_sessions.T,session_row], axis = 1).T
        dl_session_deputy = pd.concat([dl_session_deputy.T, dl_session_members.T], axis = 1, ignore_index = True).T
        df_events = pd.concat([df_events.T] + events, axis = 1).T
        df_speeches = pd.concat([df_speeches.T] + speeches, axis = 1).T
        dl_deputy_event = pd.concat([dl_deputy_event.T] + deputy_event_links, axis = 1, ignore_index = True).T
        dl_party_event = pd.concat([dl_party_event.T] + party_event_links, axis = 1, ignore_index = True).T

    return [df_deputies,df_sessions,df_events,df_speeches,dl_deputy_period,dl_session_deputy,dl_deputy_event,dl_party_event]

In [96]:
import pandas as pd
my_df = pd.DataFrame([['max','meier'],['hans','wurst']], columns = ['firstname','lastname'])
my_ds = pd.Series(['karl','mustermann'], name = 3, index = ['firstname','lastname'])#, pd.Series(['karl1','mustermann1'], name = 4, index = ['firstname','lastname'])]
#print(pd.concat([my_df.T,my_ds], axis=1).T)
print(my_df.loc[[0,1]])
my_df.to_csv('../0_datasets/integrated_data/test.csv', index = True)
#find_deputies("Max, Meier und der Wurst, Hans gingen auf die Toilette.", my_df)

#print(standardise('Merkel, Dr. Angela'))

  firstname lastname
0       max    meier
1      hans    wurst


In [15]:
[df_deputies,df_sessions,df_events,df_speeches,dl_session_deputy,dl_deputy_event,dl_party_event] = fetch_all('../0_datasets/MDB_STAMMDATEN.XML', ['../0_datasets/bundestag/plenarprotokoll_5386.xml'])
#print(df_events)
#print(df_deputies)
for index, row in df_events.iterrows():
    if(row['speech_id'] != "ID1920511800"):
        continue
    print('....Event ' + str(row['event_type']) + " in speech " + str(row['speech_id']))
    print(dl_deputy_event[dl_deputy_event['event_id'] == index])
    for index2, deputy in df_deputies.filter( items = dl_deputy_event[dl_deputy_event['event_id'] == index]['deputy_id'], axis = 0).iterrows():
        print(deputy['firstname'] + " " + deputy['lastname'])


Fetching ../0_datasets/MDB_STAMMDATEN.XML ...
CDU
FDP
CDU
CDU
CDU
SPD
CDU
CDU
SPD
CDU
CDU
KPD
SPD
DP
SPD
SPD
CSU
CDU
CDU
SPD
FDP
CDU
SPD
CDU
SPD
CDU
CSU
CDU
SPD
SPD
DZP
SPD
SPD
CDU
SPD
SPD
SPD
FDP
FDP
SPD
SPD
SPD
SPD
FU
CDU
SPD
SPD
SPD
SPD
CDU
CDU
FU
CDU
FDP
SPD
FDP
SPD
SPD
SPD
FDP
CDU
CDU
GRÜNE
BP
CDU
SPD
SPD
FDP
CDU
SPD
SPD
SPD
SPD
SPD
SPD
SPD
DP
CDU
SPD
CDU
CDU
CSU
CDU
SPD
SPD
SPD
SPD
FDP
SPD
SPD
SPD
GRÜNE
SPD
CDU
CDU
CDU
CDU
CDU
SPD
SPD
CDU
GRÜNE
SPD
SPD
CDU
CSU
CDU
CSU
CDU
FDP
CDU
BP
CDU
SPD
CDU
SPD
SPD
CSU
CDU
SPD
CSU
SPD
CDU
DP
SPD
CDU
CDU
FDP
CDU
SPD
FDP
BÜNDNIS 90/DIE GRÜNEN
SPD
SPD
SPD
CDU
CDU
GB/ BHE
CDU
SPD
CDU
CDU
CDU
CDU
FDP
SPD
CDU
CDU
CDU
SPD
FDP
SPD
CDU
CDU
SPD
SPD
SPD
CDU
SPD
SPD
SPD
FU
CDU
CSU
SPD
SPD
SPD
CDU
SPD
CDU
CDU
DP
SPD
CSU
SPD
CDU
CDU
SPD
SPD
SPD
CDU
DIE GRÜNEN/BÜNDNIS 90
CDU
CDU
CDU
PDS
PDS
SPD
SPD
CDU
FDP
CSU
CDU
SPD
CDU
CDU
CDU
CDU
CDU
DP
CDU
CDU
SPD
CDU
SPD
FDP
CDU
SPD
CDU
CDU
CSU
FDP
CDU
SPD
SPD
CDU
CDU
CDU
SPD
SPD
DZP
SPD
SPD
CDU
CSU
SPD
CDU
CDU
FDP
CD

KeyboardInterrupt: 

In [17]:
import os
import time
mdb_path = '../0_datasets/MDB_STAMMDATEN.XML'
protocol_paths = os.listdir('../0_datasets/bundestag')
protocol_paths = ['../0_datasets/bundestag/{0}'.format(path) for path in protocol_paths]
start = time.time()

[df_deputies,df_sessions,df_events,df_speeches,dl_deputy_period, dl_session_deputy,dl_deputy_event,dl_party_event] = fetch_all(mdb_path, protocol_paths)
end = time.time()
print("Total time: + " + str(end-start))


df_deputies.to_csv('../0_datasets/integrated_data/df_deputies.csv', index = True)
df_sessions.to_csv('../0_datasets/integrated_data/df_sessions.csv', index = True)
df_events.to_csv('../0_datasets/integrated_data/df_events.csv', index = True)
df_speeches.to_csv('../0_datasets/integrated_data/df_speeches.csv', index = True)
dl_deputy_period.to_csv('../0_datasets/integrated_data/dl_deputy_period.csv', index = False)
dl_session_deputy.to_csv('../0_datasets/integrated_data/dl_session_deputy.csv', index = False)
dl_deputy_event.to_csv('../0_datasets/integrated_data/dl_deputy_event.csv', index = False)
dl_party_event.to_csv('../0_datasets/integrated_data/dl_party_event.csv', index = False)

CDU
FDP
CDU
CDU
CDU
SPD
CDU
CDU
SPD
CDU
CDU
KPD
SPD
DP
SPD
SPD
CSU
CDU
CDU
SPD
FDP
CDU
SPD
CDU
SPD
CDU
CSU
CDU
SPD
SPD
DZP
SPD
SPD
CDU
SPD
SPD
SPD
FDP
FDP
SPD
SPD
SPD
SPD
FU
CDU
SPD
SPD
SPD
SPD
CDU
CDU
FU
CDU
FDP
SPD
FDP
SPD
SPD
SPD
FDP
CDU
CDU
GRÜNE
BP
CDU
SPD
SPD
FDP
CDU
SPD
SPD
SPD
SPD
SPD
SPD
SPD
DP
CDU
SPD
CDU
CDU
CSU
CDU
SPD
SPD
SPD
SPD
FDP
SPD
SPD
SPD
GRÜNE
SPD
CDU
CDU
CDU
CDU
CDU
SPD
SPD
CDU
GRÜNE
SPD
SPD
CDU
CSU
CDU
CSU
CDU
FDP
CDU
BP
CDU
SPD
CDU
SPD
SPD
CSU
CDU
SPD
CSU
SPD
CDU
DP
SPD
CDU
CDU
FDP
CDU
SPD
FDP
BÜNDNIS 90/DIE GRÜNEN
SPD
SPD
SPD
CDU
CDU
GB/ BHE
CDU
SPD
CDU
CDU
CDU
CDU
FDP
SPD
CDU
CDU
CDU
SPD
FDP
SPD
CDU
CDU
SPD
SPD
SPD
CDU
SPD
SPD
SPD
FU
CDU
CSU
SPD
SPD
SPD
CDU
SPD
CDU
CDU
DP
SPD
CSU
SPD
CDU
CDU
SPD
SPD
SPD
CDU
DIE GRÜNEN/BÜNDNIS 90
CDU
CDU
CDU
PDS
PDS
SPD
SPD
CDU
FDP
CSU
CDU
SPD
CDU
CDU
CDU
CDU
CDU
DP
CDU
CDU
SPD
CDU
SPD
FDP
CDU
SPD
CDU
CDU
CSU
FDP
CDU
SPD
SPD
CDU
CDU
CDU
SPD
SPD
DZP
SPD
SPD
CDU
CSU
SPD
CDU
CDU
FDP
CDU
GRÜNE
FDP
CDU
SPD
SPD
PDS/LL
CDU
SPD
CDU
CDU


KeyboardInterrupt



In [2]:
mdb_path = '../0_datasets/MDB_STAMMDATEN.XML'
[df_deputies, dl_deputy_period] = load_deputies_from_xml(mdb_path)
df_deputies.to_csv('../0_datasets/integrated_data/df_deputies.csv', index = True)

CDU
FDP
CDU
CDU
CDU
SPD
CDU
CDU
SPD
CDU
CDU
KPD
SPD
DP
SPD
SPD
CSU
CDU
CDU
SPD
FDP
CDU
SPD
CDU
SPD
CDU
CSU
CDU
SPD
SPD
DZP
SPD
SPD
CDU
SPD
SPD
SPD
FDP
FDP
SPD
SPD
SPD
SPD
FU
CDU
SPD
SPD
SPD
SPD
CDU
CDU
FU
CDU
FDP
SPD
FDP
SPD
SPD
SPD
FDP
CDU
CDU
GRÜNE
BP
CDU
SPD
SPD
FDP
CDU
SPD
SPD
SPD
SPD
SPD
SPD
SPD
DP
CDU
SPD
CDU
CDU
CSU
CDU
SPD
SPD
SPD
SPD
FDP
SPD
SPD
SPD
GRÜNE
SPD
CDU
CDU
CDU
CDU
CDU
SPD
SPD
CDU
GRÜNE
SPD
SPD
CDU
CSU
CDU
CSU
CDU
FDP
CDU
BP
CDU
SPD
CDU
SPD
SPD
CSU
CDU
SPD
CSU
SPD
CDU
DP
SPD
CDU
CDU
FDP
CDU
SPD
FDP
BÜNDNIS 90/DIE GRÜNEN
SPD
SPD
SPD
CDU
CDU
GB/ BHE
CDU
SPD
CDU
CDU
CDU
CDU
FDP
SPD
CDU
CDU
CDU
SPD
FDP
SPD
CDU
CDU
SPD
SPD
SPD
CDU
SPD
SPD
SPD
FU
CDU
CSU
SPD
SPD
SPD
CDU
SPD
CDU
CDU
DP
SPD
CSU
SPD
CDU
CDU
SPD
SPD
SPD
CDU
DIE GRÜNEN/BÜNDNIS 90
CDU
CDU
CDU
PDS
PDS
SPD
SPD
CDU
FDP
CSU
CDU
SPD
CDU
CDU
CDU
CDU
CDU
DP
CDU
CDU
SPD
CDU
SPD
FDP
CDU
SPD
CDU
CDU
CSU
FDP
CDU
SPD
SPD
CDU
CDU
CDU
SPD
SPD
DZP
SPD
SPD
CDU
CSU
SPD
CDU
CDU
FDP
CDU
GRÜNE
FDP
CDU
SPD
SPD
PDS/LL
CDU
SPD
CDU
CDU